In [1]:
from pycaret.datasets import get_data
import pycaret.regression as reg

boston = get_data('boston')
session = reg.setup(data=boston, target='medv', silent=True, verbose=False)

topk = reg.compare_models(n_select=3, include=['rf', 'gbr', 'et'])
topk_tuned = [reg.tune_model(model) for model in topk]

bagger = [reg.ensemble_model(model, method='Bagging') for model in topk_tuned]
booster = [reg.ensemble_model(model, method='Boosting') for model in topk_tuned]
blender = reg.blend_models(topk_tuned)
stacker = reg.stack_models(topk_tuned)

best_automl = reg.automl(optimize='MAE')
best_automl = reg.finalize_model(best_automl)

best_model = reg.get_config('prep_pipe')
best_model.steps.append(['trained_model', best_automl])
print(">>", type(best_model.steps[-1][-1]))

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,1.7991,4.5741,2.1387,0.9377,0.1097,0.0972
1,2.5859,15.2156,3.9007,0.8602,0.1731,0.1456
2,2.0876,8.7829,2.9636,0.8803,0.1421,0.1013
3,1.7019,5.4039,2.3246,0.9318,0.1152,0.0886
4,2.6142,18.8052,4.3365,0.8100,0.1321,0.0934
5,2.0157,6.5280,2.5550,0.8630,0.1308,0.1021
6,2.0618,9.8248,3.1345,0.8806,0.1379,0.1010
7,2.5305,15.4367,3.9290,0.8246,0.1599,0.1270
8,2.0615,7.7093,2.7766,0.8829,0.1606,0.1141


>> <class 'sklearn.ensemble._forest.ExtraTreesRegressor'>


In [2]:
best_model.steps[:-1]

[('dtypes',
  DataTypes_Auto_infer(categorical_features=[], display_types=False,
                       features_todrop=[], id_columns=[], ml_usecase='regression',
                       numerical_features=[], target='medv', time_features=[])),
 ('imputer',
  Simple_Imputer(categorical_strategy='not_available',
                 fill_value_categorical=None, fill_value_numerical=None,
                 numeric_strategy='mean', target_variable=None)),
 ('new_levels1',
  New_Catagorical_Levels_in_TestData(replacement_strategy='least frequent',
                                     target='medv')),
 ('ordinal', 'passthrough'),
 ('cardinality', 'passthrough'),
 ('znz', 'passthrough'),
 ('club_R_L', 'passthrough'),
 ('new_levels',
  New_Catagorical_Levels_in_TestData(replacement_strategy='least frequent',
                                     target='medv')),
 ('feature_time',
  Make_Time_Features(list_of_features=None,
                     time_feature=Index([], dtype='object'))),
 ('group', 'p

In [3]:
best_model.steps[-1]

['trained_model',
 ExtraTreesRegressor(bootstrap=False, ccp_alpha=0.0, criterion='mse',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     max_samples=None, min_impurity_decrease=0.0,
                     min_impurity_split=None, min_samples_leaf=1,
                     min_samples_split=2, min_weight_fraction_leaf=0.0,
                     n_estimators=100, n_jobs=-1, oob_score=False,
                     random_state=4660, verbose=0, warm_start=False)]